# Groupchat with Llamaindex agents

[Llamaindex agents](https://docs.llamaindex.ai/en/stable/optimizing/agentic_strategies/agentic_strategies/) have the ability to use planning strategies to answer user questions. They can be integrated in Autogen in easy ways

## Requirements

In [ ]:
%pip install autogen-agentchat~=0.2 llama-index llama-index-tools-wikipedia llama-index-readers-wikipedia wikipedia

## Set your API Endpoint

In [1]:
import os

import autogen

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={"tags": ["gpt-3.5-turbo"]},  # comment out to get all
)
# When using a single openai endpoint, you can use the following:
# config_list = [{"model": "gpt-3.5-turbo", "api_key": os.getenv("OPENAI_API_KEY")}]

## Set Llamaindex

In [2]:
from llama_index.core import Settings
from llama_index.core.agent import ReActAgent
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.tools.wikipedia import WikipediaToolSpec

llm = OpenAI(
    model="gpt-3.5-turbo",
    temperature=0.0,
    api_key=os.environ.get("OPENAPI_API_KEY", ""),
)

embed_model = OpenAIEmbedding(
    model="text-embedding-ada-002",
    temperature=0.0,
    api_key=os.environ.get("OPENAPI_API_KEY", ""),
)

Settings.llm = llm
Settings.embed_model = embed_model

# create a react agent to use wikipedia tool
wiki_spec = WikipediaToolSpec()
# Get the search wikipedia tool
wikipedia_tool = wiki_spec.to_tool_list()[1]

location_specialist = ReActAgent.from_tools(tools=[wikipedia_tool], llm=llm, max_iterations=10, verbose=True)

## Create agents

In this example, we will create a Llamaindex agent to answer questions fecting data from wikipedia and a user proxy agent.

In [3]:
from autogen.agentchat.contrib.llamaindex_conversable_agent import LLamaIndexConversableAgent

llm_config = {
    "temperature": 0,
    "config_list": config_list,
}

trip_assistant = LLamaIndexConversableAgent(
    "trip_specialist",
    llama_index_agent=location_specialist,
    system_message="You help customers finding more about places they would like to visit. You can use external resources to provide more details as you engage with the customer.",
    description="This agents helps customers discover locations to visit, things to do, and other details about a location. It can use external resources to provide more details. This agent helps in finding attractions, history and all that there si to know about a place",
)

user_proxy = autogen.UserProxyAgent(
    name="Admin",
    human_input_mode="ALWAYS",
    code_execution_config=False,
)

Next, let's set up our group chat.

In [4]:
groupchat = autogen.GroupChat(
    agents=[trip_assistant, user_proxy],
    messages=[],
    max_round=500,
    speaker_selection_method="round_robin",
    enable_clear_history=True,
)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

In [5]:
chat_result = user_proxy.initiate_chat(
    manager,
    message="""
What can i find in Tokyo related to Hayao Miyazaki and its moveis like Spirited Away?.
""",
)

Admin (to chat_manager):


What can i find in Tokyo related to Hayao Miyazaki and its moveis like Spirited Away?.


--------------------------------------------------------------------------------

Next speaker: trip_specialist


>>>>>>>> USING AUTO REPLY...
> Running step 4f4f291b-5e13-495f-9871-4207e4c4bcb9. Step input: 
What can i find in Tokyo related to Hayao Miyazaki and its moveis like Spirited Away?.

Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: search_data
Action Input: {'query': 'Hayao Miyazaki Tokyo'}
Observation: Hayao Miyazaki (宮崎 駿 or 宮﨑 駿, Miyazaki Hayao, Japanese: [mijaꜜzaki hajao]; born January 5, 1941) is a Japanese animator, filmmaker, and manga artist. A founder of Studio Ghibli, he has attained international acclaim as a masterful storyteller and creator of Japanese animated feature films, and is widely regarded as one of the most accomplished filmmakers in the history of animation.
Born in Toky